In [17]:
from crewai import Agent, LLM

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, LLM, Task, Crew, Process

# Load environment variables
load_dotenv()

# Option D: Azure OpenAI
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

llm = LLM(
    model="azure/gpt-4o-mini",
    api_key=openai_api_key,
    base_url=openai_endpoint,
    api_version=openai_api_version,
    azure=True
)

print("🚀 Environment configured!")
print(f"LLM Model: {llm.model}")

## Guardrails Demo 1

In [19]:
def validate_summary_length(task_output):
    try:
        print("Validating summary length")
        task_str_output = str(task_output)
        total_words = len(task_str_output.split())

        print(f"Word count: {total_words}")

        if total_words > 150:
            print("Summary exceeds 150 words")
            return (False, f"Summary exceeds 150 words. Word count: {total_words}")

        if total_words == 0:
            print("Summary is empty")
            return (False, "Generated summary is empty.")

        print("Summary is valid")
        return (True, task_output)

    except Exception as e:
        print("Validation system error")
        return (False, f"Validation system error: {str(e)}")


In [20]:
from crewai import Task, Agent

summary_agent = Agent(
    role="Summary Agent",
    goal="Summarize the research paper 'Convolutional Neural Networks' in 150 words.",
    backstory="You are a specialized agent that summarizes research papers.",
    llm=llm,
    verbose=True
)

summary_task = Task(
    description="Summarize a research paper in 150 words.",
    expected_output="A concise research summary 150 words.",
    agent=summary_agent,
    guardrail=validate_summary_length,
    
    max_retries=3
)


In [ ]:
from crewai import Crew

summary_crew = Crew(
    agents=[summary_agent],
    tasks=[summary_task],
    verbose=True
)

result = summary_crew.kickoff()

### Ask the agent to write a 200 word summary instead and notice the guardrail output:

In [22]:
from crewai import Task, Agent

summary_agent = Agent(
    role="Summary Agent",
    goal="Summarize the research paper 'Convolutional Neural Networks' in 200 words.",
    backstory="You are a specialized agent that summarizes research papers.",
    llm=llm,
    verbose=True
)

summary_task = Task(
    description="Summarize a research paper in 200 words.",
    expected_output="A concise research summary 200 words.",
    agent=summary_agent,
    guardrail=validate_summary_length,
    max_retries=3
)


In [ ]:
from crewai import Crew

summary_crew = Crew(
    agents=[summary_agent],
    tasks=[summary_task],
    verbose=True
)

result = summary_crew.kickoff()

## Guardrails demo 2

In [24]:
from pydantic import BaseModel

class ResearchReport(BaseModel):
    """Represents a structured research report"""
    title: str
    summary: str
    key_findings: list[str]

In [25]:
import json
from typing import Tuple, Any

def validate_json_report(result: str) -> Tuple[bool, Any]:
    """Ensures AI-generated output is valid JSON with required fields."""
    try:
        # Parse JSON output
        data = json.loads(result.pydantic.model_dump_json())

        # Check required fields
        if "title" not in data or "summary" not in data or "key_findings" not in data:
            return (False, "Missing required fields: title, summary, or key_findings.")

        return (True, result)  # JSON is valid
    except json.JSONDecodeError:
        return (False, "Invalid JSON format. Please ensure correct syntax.")


In [26]:
from crewai import Agent

# Create the AI Agent
research_report_agent = Agent(
    role="Research Analyst",
    goal="Generate structured JSON reports for research papers",
    backstory="You are an expert in technical writing and structured reporting.",
    llm=llm,
    verbose=False,
)


In [27]:
from crewai import Task

research_report_task = Task(
    description="Generate a structured research report in valid JSON format.",
    expected_output="A valid JSON object containing 'title', 'summary', and 'key_findings'.",
    agent=research_report_agent,
    output_pydantic=ResearchReport,  # Ensures structured output
    guardrail=validate_json_report,  # Validate output before passing to next step
    max_retries=3  # Allow up to 3 retries if validation fails
)

In [28]:
from crewai import Crew

research_crew = Crew(
    agents=[research_report_agent],
    tasks=[research_report_task],
    verbose=True  # Display execution details
)


In [ ]:
result = research_crew.kickoff()

# Display the validated JSON output
print("Final Research Report:", result.pydantic)